# Loading IRIS Dataset

In [1]:
from sklearn.datasets import load_iris
iris = load_iris()

print(iris.data) 
print(iris.target)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

# Importing Libraries

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import time


# Defining Functions

In [ ]:
# Gaussian RBF function
def rbf(x, center, sigma):
    return np.exp(-np.linalg.norm(x - center, axis=1)**2 / (2 * sigma**2))

# Train the RBF Network
def train_rbf_network(X_train, y_train, centers, sigma):
    # Compute the RBF layer output
    rbf_outputs = np.array([rbf(X_train, center, sigma) for center in centers]).T
    # Add bias term
    rbf_outputs = np.hstack((rbf_outputs, np.ones((rbf_outputs.shape[0], 1))))
    # Solve weights using linear regression
    weights = np.linalg.pinv(rbf_outputs) @ y_train
    return weights

# Predict using the RBF Network
def predict_rbf_network(X, centers, weights, sigma):
    rbf_outputs = np.array([rbf(X, center, sigma) for center in centers]).T
    rbf_outputs = np.hstack((rbf_outputs, np.ones((rbf_outputs.shape[0], 1))))
    predictions = rbf_outputs @ weights
    return np.argmax(predictions, axis=1)
## this part of code was generated with the aid of A.I.


# Running the Code

In [14]:
# Load the Iris dataset
data = load_iris()
X, y = data.data, data.target
y = OneHotEncoder(sparse_output=False).fit_transform(y.reshape(-1, 1))  # One-hot encode target

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

sigma = 1.0

for num_centers in range(1, 25):
    
    # Method 1 - Random Selection of Centers
    start_time_random = time.time()
    # Choosing some random elements of the current dataset as the centers for RBF
    random_indices = np.random.choice(X_train.shape[0], num_centers, replace=False)
    random_centers = X_train[random_indices]
    weights_random = train_rbf_network(X_train, y_train, random_centers, sigma)
    y_pred_random = predict_rbf_network(X_test, random_centers, weights_random, sigma)
    accuracy_random = accuracy_score(np.argmax(y_test, axis=1), y_pred_random)
    end_time_random = time.time()
    time_random = end_time_random - start_time_random 
    # Results of Randomly Selected Centers
    print("Accuracy with Random Selection of ", num_centers, " Centers:", accuracy_random)
    print("Time taken for Random Selection:", time_random, "seconds")

    # Method 2: K-Means Clustering for Centers
    start_time_kmeans = time.time()  # Start the timer
    kmeans = KMeans(n_clusters=num_centers, random_state=42, n_init=10)
    kmeans.fit(X_train)
    kmeans_centers = kmeans.cluster_centers_
    weights_kmeans = train_rbf_network(X_train, y_train, kmeans_centers, sigma)
    y_pred_kmeans = predict_rbf_network(X_test, kmeans_centers, weights_kmeans, sigma)
    accuracy_kmeans = accuracy_score(np.argmax(y_test, axis=1), y_pred_kmeans)
    end_time_kmeans = time.time() 
    time_kmeans = end_time_kmeans - start_time_kmeans

    print("Accuracy with K-Means Clustering for ", num_centers," Centers:", accuracy_kmeans)
    print("Time taken for K-Means Clustering:", time_kmeans, "seconds")

    print("K-Means takes ", time_kmeans / time_random , " times more than random")
    print('=' * 65)


Accuracy with Random Selection of  1  Centers: 0.6222222222222222
Time taken for Random Selection: 0.0012700557708740234 seconds
Accuracy with K-Means Clustering for  1  Centers: 0.7555555555555555
Time taken for K-Means Clustering: 0.0060575008392333984 seconds
K-Means takes  4.769476253050497  times more than random
Accuracy with Random Selection of  2  Centers: 0.8888888888888888
Time taken for Random Selection: 0.001049041748046875 seconds
Accuracy with K-Means Clustering for  2  Centers: 0.7333333333333333
Time taken for K-Means Clustering: 0.010837078094482422 seconds
K-Means takes  10.330454545454545  times more than random
Accuracy with Random Selection of  3  Centers: 0.8444444444444444
Time taken for Random Selection: 0.0012540817260742188 seconds
Accuracy with K-Means Clustering for  3  Centers: 0.8222222222222222
Time taken for K-Means Clustering: 0.014787673950195312 seconds
K-Means takes  11.791634980988594  times more than random
Accuracy with Random Selection of  4  Cen